In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [38]:
csidf = pd.read_excel("subcarriers_amp_pos_pot.xlsx", sheet_name="subcarriers_amp_pos_pot")
csidf.drop("Unnamed: 0", axis=1, inplace=True)
csidf.head()

,device,local,x,y,P_MW_LLTF,P_MW_HTLTF,P_DBM_LLTF,P_DBM_HTLTF,LLTF_-32,LLTF_-31,...,HTLTF_22,HTLTF_23,HTLTF_24,HTLTF_25,HTLTF_26,HTLTF_27,HTLTF_28,HTLTF_29,HTLTF_30,HTLTF_31
0,DIR,A1,1,3,68,1763,18.326305,32.462711,0,0,...,22.090722,19.000000,17.029386,15.132746,14.560220,12.806248,13.601471,2.236068,0.000000,1.414214
1,DIR,A1,1,3,91,2902,19.625982,34.627993,0,0,...,24.083189,22.203603,19.416488,17.720045,17.464249,15.620499,15.000000,3.605551,2.236068,0.000000
2,DIR,A1,1,3,79,1819,19.009755,32.599737,0,0,...,21.023796,18.110770,17.464249,15.811388,14.317821,13.453624,12.806248,2.236068,1.000000,1.414214
3,DIR,A1,1,3,82,2482,19.157725,33.949590,0,0,...,21.954498,19.416488,17.029386,15.556349,15.000000,12.649111,12.041595,2.000000,0.000000,1.000000
4,DIR,A1,1,3,43,736,16.433430,28.670187,0,0,...,20.000000,17.492856,15.231546,13.601471,13.152946,13.038405,12.369317,2.236068,1.000000,1.414214


In [39]:
def dbmtopot(dbm):
    return 10**(dbm/10)

In [40]:
csidf[csidf.columns[8:]] = csidf[csidf.columns[8:]].apply(dbmtopot)

In [42]:
csidf.head()

,device,local,x,y,P_MW_LLTF,P_MW_HTLTF,P_DBM_LLTF,P_DBM_HTLTF,LLTF_-32,LLTF_-31,...,HTLTF_22,HTLTF_23,HTLTF_24,HTLTF_25,HTLTF_26,HTLTF_27,HTLTF_28,HTLTF_29,HTLTF_30,HTLTF_31
0,DIR,A1,1,3,68,1763,18.326305,32.462711,1.0,1.0,...,161.834907,79.432823,50.459000,32.604279,28.577352,19.082042,22.916435,1.673427,1.000000,1.384909
1,DIR,A1,1,3,91,2902,19.625982,34.627993,1.0,1.0,...,256.046543,166.096443,87.427646,59.156778,55.773117,36.479589,31.622777,2.293798,1.673427,1.000000
2,DIR,A1,1,3,79,1819,19.009755,32.599737,1.0,1.0,...,126.584230,64.725740,55.773117,38.118766,27.026021,22.149422,19.082042,1.673427,1.258925,1.384909
3,DIR,A1,1,3,82,2482,19.157725,33.949590,1.0,1.0,...,156.837474,87.427646,50.459000,35.944705,31.622777,18.403951,16.001454,1.584893,1.000000,1.258925
4,DIR,A1,1,3,43,736,16.433430,28.670187,1.0,1.0,...,100.000000,56.141701,33.354514,22.916435,20.667819,20.129847,17.255664,1.673427,1.258925,1.384909


In [44]:
#Divisão dos dados em conjunto de treino e teste
from sklearn.model_selection import train_test_split #Importa a função de split

In [45]:
#Usando LLTF apenas e não considerando a direção - Multiplas vezes
total_repeticoes = 10
colunas = csidf.columns[8:70]
locais = sorted(csidf["local"].unique())
percent_acertos_df = pd.DataFrame(columns=locais,index=range(total_repeticoes))
percent_acertos_df["Total"]=0.00
for n in range(total_repeticoes):
    aux = 0
    for k in csidf["local"].unique():
        for j in csidf["device"].unique():
            data_y = csidf[(csidf["local"]==k) & (csidf["device"]==j)].copy()
            datalltf= data_y[colunas]
            if aux==0:
                X_train, X_test, y_train, y_test = train_test_split(datalltf,data_y["local"],test_size=0.2)
                aux=aux+1
            else:
                X_train_aux, X_test_aux, y_train_aux, y_test_aux = train_test_split(datalltf,data_y["local"],test_size=0.2)
                X_train = X_train.append(X_train_aux)
                X_test = X_test.append(X_test_aux)
                y_train = y_train.append(y_train_aux)
                y_test = y_test.append(y_test_aux)
    minmeansquareerror = []
    best_index_match = []
    best_atual_index = 0
    menor_mse = 10000
    mse_atual = 0
    for k  in X_test.index:
    # k = X_test.index[3]
        menor_mse = 10000
        best_atual_index = 0
        for j in X_train.index:
            mse_atual = 0
            for w in X_train.columns:
                mse_atual += (X_train[w][j] - X_test[w][k])**(2)
            if mse_atual < menor_mse:
                menor_mse = mse_atual
                best_atual_index = j
        minmeansquareerror.append(menor_mse)
        best_index_match.append(best_atual_index)
    # print(minmeansquareerror)
    # print(best_index_match)
    # print("A localização real é:" + y_test[k])
    # print("A resposta do algoritmo deu:" + y_train[best_index_match])
    #GERANDO UMA TABELA DE COMPARAÇÃO
    cols = ["index","real_location","estimated_location","acertou"]
    compare_table = pd.DataFrame(columns=cols)
    compare_table["index"] = y_test.index
    for x in range(len(compare_table)):
        compare_table["real_location"][x]=y_test[y_test.index[x]]

    x=0
    for i in best_index_match:
        compare_table["estimated_location"][x] = y_train[i]
        if(compare_table["estimated_location"][x] == compare_table["real_location"][x]):
            compare_table["acertou"][x]=1
        else:
            compare_table["acertou"][x]=0
        x=x+1

    for h in locais:
        acertos = len(compare_table[(compare_table["real_location"]==h) & (compare_table["acertou"]==1)])/len(compare_table[compare_table["real_location"]==h])
        percent_acertos_df[h][n]=acertos
    total = round((((len(compare_table[compare_table["acertou"]==1])/(len(compare_table))))*100),2)
    percent_acertos_df["Total"][n]= total
    print("Total de acerto da repeticao "+ str(n) +": " + str(total) + "%")

C:\Users\iande\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iande\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iande\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iande\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

Total de acerto da repeticao 0: 10.42%


KeyError: 0